# 5. DOMAČA NALOGA

Pri tej domači nalogi boste rešili nekaj računskih problemov s pomočjo metod, ki smo jih spoznali na predavanjih. Vse naloge rešujte v OCamlu, vsako rešitev pa dobro dokumentirajte, da bo iz nje razvidna pravilnost ter časovna zahtevnost rešitve.

Pri vsaki nalogi bomo ocenjevali učinkovitost, preglednost in eleganco rešitve ter natančnost, razumljivost in pravilnost spremnega besedila.

## DELI IN VLADAJ 
Rešiti morate dva klasična problema, ki se rešujeta z metodo deli in vladaj.

### Hitro izbiranje
Dan naj bo neurejen seznam celih števil $[a_1, a_2, ... , a_n]$ in število $1 \le k \le n$. Napišite algoritem, ki v času $O(n)$ poišče po velikosti $k$-to število v seznamu. Torej za $k = 1$ bi algoritem vrnil najmanjše, za $k = n$ pa največje število v seznamu.

In [1]:
(*najprej definiram funkcijo particija, ki za vhodne podatke vzame pivot, seznam ki ga želimo razdeliti ter v začetku dva prazna seznama, 
označena l in d, ki se nato z rekurzivnimi klici funkcije polnita, dokler ne postaneta particiji prvotnega seznama*)

(*particija je pomožna funkcija algoritmu hitrega izbiranja*)

let rec particija pivot lst l d = match lst with
  (*če pridem do praznega seznama, je postopek particije končan in vrnem do tedaj pridelana seznama*)
  | [] -> (l, d)
  (*sicer pa vsak element v seznamu primerjam s pivotom*)
  | x :: xs -> if x < pivot then
    (*če je strogo manjši od pivota, ga dam v levi seznam in rekurzivno kličem naprej*)
    particija pivot xs (x :: l) d
  else
    (*sicer pa ga dam v desni seznam*)
    particija pivot xs l (x :: d)

val particija : 'a -> 'a list -> 'a list -> 'a list -> 'a list * 'a list =
  <fun>


In [2]:
let test1 = particija 9 [10; 12; 6; 9; 3; 5; 1; 19; 16; 8; 2] [] []

val test1 : int list * int list = ([2; 8; 1; 5; 3; 6], [16; 19; 9; 12; 10])


In [3]:
(*algoritem hitro_izbiranje / quick_select*)

let rec hitro_izbiranje lst k = match lst with
(*vhodna podatka: seznam nesortiranih števil, indeks k*)
  (*dva robna primera: prazen seznam in seznam s samo enim elementom - vrne ta element*)
  | [] -> failwith "Seznam je prazen."
  | [x] -> x 

  (*sicer pa v vsakem koraku izbere pivot, ki je prvi element iz seznama*)
  | pivot :: xs -> 
    (*na podlagi tega pivota ustvari particijo z zgoraj spisano funkcijo*)
    let (l, d) = particija pivot xs [] [] in
    let len_l = List.length l in
    (*preverim, v katero particijo pade indeks k*)
    if k - 1 < len_l then
      (*če je v levi particiji, rekurzivno kličem naprej z novim (levim) seznamom in nespremenjenim indeksom*)
      hitro_izbiranje l k 
    else if k - 1 > len_l then
      (*če spada v desno particijo pa rekurzivno kličem na desnem seznamu; tu pa je potrebno prilagoditi tudi indeks*)
      hitro_izbiranje d (k - len_l - 1)
    else
      pivot

val hitro_izbiranje : 'a list -> int -> 'a = <fun>


In [4]:
let test2 = hitro_izbiranje [3; 5; 2; 8] 3

let test3 = hitro_izbiranje [13; 22; 10; 6; 9; 3; 5; 8; 7; 19; 1] 6

let test4 = hitro_izbiranje [11; 1; 3; 8; 9; 15; 14; 2] 6

let test5 = hitro_izbiranje [1; 2; 3; 4; 5; 6; 7; 8; 9; 20; 19; 18; 17; 16; 15; 14; 13; 12; 11; 10] 10

let robni_primer1 = hitro_izbiranje [3] 2

val test2 : int = 5


val test3 : int = 8


val test4 : int = 11


val test5 : int = 10


val robni_primer1 : int = 3


##### Komentar na časovno zahtevnost:
- funkcija $particija$ za vhodna podatka sprejme pivot in seznam celih števil, in ta seznam razdeli na dva seznama. Pri tem se sprehodi čez vsak element v seznamu in ga po velikosti primerja s pivotom. Ker ima primerjava po velikosti in dodajanje elementa v seznam konstantno časovno zahtevnost $O(1)$, je zato časovna zahtevnost celotne funkcije $particija \ O(n)$, kjer je $n$ seveda dolžina vhodnega seznama 
- v funkciji $hitro\_izbiranje$ po procesu particije funkcija rekurzivno kliče sama sebe. V povprečju pa vsak naslednji korak particije razpolovi velikost problema, zato je potrebnih logaritemsko mnogo število particij (v povprečju). Ker pa je sam postopek particije časovne zahtevnosti $O(n)$, je zato tudi $hitro\_izbiranje$ iste časovne zahtevnosti. V najslabšem primeru (ko je izbran pivot vedno največji ali najmanjši element v seznamu) je potrebnih $n$ korakov particije in zato postane skupna časovna zahtevnost $O(n^2)$.

### Najbližji par točk v ravnini
Dan naj bo seznam točk $[(x_1, y_1), (x_2, y_2), ..., (x_n, y_n)]$ v ravnini. Napišite algoritem, ki v času $O(n \log n)$ izračuna par  točk $(x_i, y_i)$ in $(x_j, y_j)$, ki sta si med vsemi točkami v seznamu najbližje.

V bistvu iščem točki, ki ustrezata rešitvi funkcije: $$\min_{\substack{i, j \\ i \neq j}} \sqrt{(x_i - x_j)^2 + (y_i - y_j)^2}.$$


Najprej definiram tip $tocka$, v katerem bom zapisoval točke. Definiram še funkcijo $razdalja$, ki bo računala razdaljo med dvema točkama.

In [5]:
type tocka = {
  x: float;
  y: float
}

let razdalja t1 t2 = 
  let dx = t1.x -. t2.x in
  let dy = t1. y -. t2.y in
  sqrt(dx *. dx +. dy *. dy)

type tocka = { x : float; y : float; }


val razdalja : tocka -> tocka -> float = <fun>


Spodaj sta zapisani funkciji $hitro\_sortiranje\_x$ in $hitro\_sortiranje\_y$, ki z uporabo algoritma $quick\_sort$ uredita seznam točk po velikosti $x$ ali $y$ - koordinat. Funkciji sta časovne zahtevnosti $O(n \cdot logn)$.

In [6]:
let rec hitro_sortiranje_x lst = match lst with
  | [] -> []
  | pivot :: xs ->
    (*l...seznam z elementi, ki so manjši od pivota*)
    let l = List.filter (fun p -> p.x < pivot.x) xs in
    (*d... -||-, ki so večji od pivota*)
    let d = List.filter (fun p -> p.x >= pivot.x) xs in
    (*rekurzivno se seznama l in d urejata dalje, vsak s svojim novim pivotom*)
    (hitro_sortiranje_x l) @ [pivot] @ (hitro_sortiranje_x d)
    (*s stikanjem seznamov dobim končen urejen seznam*)

val hitro_sortiranje_x : tocka list -> tocka list = <fun>


In [7]:
let rec hitro_sortiranje_y lst = match lst with
  | [] -> []
  | pivot :: xs ->
    let l = List.filter (fun p -> p.y < pivot.y) xs in
    let d = List.filter (fun p -> p.y >= pivot.y) xs in
    (hitro_sortiranje_y l) @ [pivot] @ (hitro_sortiranje_y d)

val hitro_sortiranje_y : tocka list -> tocka list = <fun>


In [8]:
(*primer seznama točk, ki ga bom uporabljal za izračun min razdalje*)
let tocke = [
  { x = 1.5; y = 3.2 };
  { x = 4.8; y = 1.9 };
  { x = 2.3; y = 4.7 };
  { x = 6.1; y = 2.8 };
  { x = 3.4; y = 5.6 };
  { x = 7.2; y = 3.3 };
  { x = 5.5; y = 6.7 };
  { x = 8.0; y = 4.1 };
  { x = 9.3; y = 7.5 }
]

let primer1 = hitro_sortiranje_x tocke
let primer2 = hitro_sortiranje_y tocke

val tocke : tocka list =
  [{x = 1.5; y = 3.2}; {x = 4.8; y = 1.9}; {x = 2.3; y = 4.7};
   {x = 6.1; y = 2.8}; {x = 3.4; y = 5.6}; {x = 7.2; y = 3.3};
   {x = 5.5; y = 6.7}; {x = 8.; y = 4.1}; {x = 9.3; y = 7.5}]


val primer1 : tocka list =
  [{x = 1.5; y = 3.2}; {x = 2.3; y = 4.7}; {x = 3.4; y = 5.6};
   {x = 4.8; y = 1.9}; {x = 5.5; y = 6.7}; {x = 6.1; y = 2.8};
   {x = 7.2; y = 3.3}; {x = 8.; y = 4.1}; {x = 9.3; y = 7.5}]


val primer2 : tocka list =
  [{x = 4.8; y = 1.9}; {x = 6.1; y = 2.8}; {x = 1.5; y = 3.2};
   {x = 7.2; y = 3.3}; {x = 8.; y = 4.1}; {x = 2.3; y = 4.7};
   {x = 3.4; y = 5.6}; {x = 5.5; y = 6.7}; {x = 9.3; y = 7.5}]


Sedaj bom definiral funkcijo, ki razdeli seznam urejen naraščajoče po $x$ - koordinatah na polovico in vrne oba seznama. Funkcija bo uporabna v nadaljevanju, ko bom v seznamih iskal minimalno razdaljo.

In [13]:
let razdeli_tocke lst = 
  let k = (List.length lst) / 2 in
  let rec aux i acc1 acc2 = function
  (*acc1 in acc2 sta seznama elementov levo in desno od sredinskega*)
  | [] -> (List.rev acc1, List.rev acc2)
  | x :: xs -> 
    if i < k then
      aux (i + 1) (x :: acc1) acc2 xs
    else
      aux (i + 1) acc1 (x :: acc2) xs 
  in
  aux 0 [] [] lst

val razdeli_tocke : 'a list -> 'a list * 'a list = <fun>


In [14]:
let primer3 = razdeli_tocke primer1

val primer3 : tocka list * tocka list =
  ([{x = 1.5; y = 3.2}; {x = 2.3; y = 4.7}; {x = 3.4; y = 5.6};
    {x = 4.8; y = 1.9}],
   [{x = 5.5; y = 6.7}; {x = 6.1; y = 2.8}; {x = 7.2; y = 3.3};
    {x = 8.; y = 4.1}; {x = 9.3; y = 7.5}])


Funkcija $min\_razdalja$ izračuna najmanjšo razdaljo v obeh seznamih.

In [16]:
let rec min_razdalja (tocke1, tocke2) = match tocke1, tocke2 with
  (*če je kateri od seznamov prazen, je razdalja med njima neskončna*)
  | [], _ -> infinity  
  | _, [] -> infinity  
  (*sicer pa rekurzivno računam najmanjšo razdaljo*)
  | t1 :: ostalo1, t2 :: ostalo2 ->
    let t1_t2 = razdalja t1 t2 in
    let min_d = min (min_razdalja (ostalo1, tocke2)) (min_razdalja (tocke1, ostalo2)) in
    min t1_t2 min_d

val min_razdalja : tocka list * tocka list -> float = <fun>


In [17]:
let primer4 = min_razdalja primer3

val primer4 : float = 1.58113883008418932


Sedaj definiram funkcijo, ki vrne sredinsko točko v seznamu urejenem po x - koordinatah.

In [18]:
let sredinska_tocka lst = 
  let urejen_seznam = hitro_sortiranje_x lst in
  let len = List.length urejen_seznam in
  List.nth urejen_seznam (len / 2)

val sredinska_tocka : tocka list -> tocka = <fun>


In [19]:
let primer5 = sredinska_tocka tocke

val primer5 : tocka = {x = 5.5; y = 6.7}


Funkcija $strip$ vrne seznam točk, ki so po $x$ - koordinatah od sredinske točke oddaljene za manj ali enako od prej izračunane najmanjše razdalje v obeh seznamih.

In [20]:
let rec strip lst d = 
  let sredina = sredinska_tocka lst in
  hitro_sortiranje_y (List.filter (fun tocka -> abs_float(tocka.x -. sredina.x) <= d) lst)

val strip : tocka list -> float -> tocka list = <fun>


In [21]:
let primer6 = strip primer1 primer4

val primer6 : tocka list =
  [{x = 4.8; y = 1.9}; {x = 6.1; y = 2.8}; {x = 5.5; y = 6.7}]


Nazadnje poračunam najmanjšo razdlajo v seznamu strip.

In [23]:
let najblizji_par lst =
  let rec najmanjsa_razdalja min_par min_dist = function
    | [] | [_] -> min_par, min_dist
    | t1 :: ostalo ->
      let nov_min_par, nova_min_dist =
        List.fold_left (fun (par, dist) t2 ->
          let d = razdalja t1 t2 in
          if d < dist then (t1, t2), d else par, dist) (min_par, min_dist) ostalo
      in
      najmanjsa_razdalja nov_min_par nova_min_dist ostalo
  in
  match lst with
  | [] | [_] -> failwith "prekratek seznam"
  | t1 :: t2 :: ostalo -> najmanjsa_razdalja (t1, t2) (razdalja t1 t2) (t1 :: t2 :: ostalo)


val najblizji_par : tocka list -> (tocka * tocka) * float = <fun>


In [24]:
let primer7 = najblizji_par primer6

val primer7 : (tocka * tocka) * float =
  (({x = 4.8; y = 1.9}, {x = 6.1; y = 2.8}), 1.58113883008418932)


## DINAMIČNO PROGRAMIRANJE
Iz spodnjega seznama nalog na strani $Project\ Euler$ rešite naloge, skupaj vredne vsaj 6 točk:
- <span style="color:green"> **#31 Coin Sums (1)**</span>
- #67 Maximum Path Sum II (1)
- #82 Path Sum: Three Ways (2)
- #115 Counting block Combinations II (2)

- <span style="color:green"> **#117 Red, Green, and Blue Tiles (2)**</span>

- #215 Vrack-free walls (3)
- #534 Weak Queens (4)

#### #31 Coin Sums
Pri reševanju te naloge je ključna rekurzivna formula $$st\_kombinacij(kovanci, n, sum) = st\_kombinacij(kovanci, n, sum - kovanci[n-1]) + st\_kombinacij(kovanci, n - 1, sum)$$ Rekurzivna formula pravi, da lahko število vseh kombinaicij razdelimo na vsoto dveh dogodkov; ko smo določen kovanec izbrali, ali pa ko ga nismo izbrali. Če smo ta kovanec izbrali, njegovo vrednost odštejemo od $sum$, če pa ne, v rekurzivni formuli zmanjšamo število kovancev za 1. Zelo sta pomembna še robna pogoja $$st\_kombinacij(kovanci, n, 0) = 1$$ ter $$st\_kombinacij(kovanci, 0, sum) = 0.$$ Prvi robni pogoj pravi, da ko je vrednost $sum$ enaka 0, obstaja le še en način, da sestavimo to vrednost in sicer tako, da ne izberemo nobenega kovanca. Drugi pogoj pa nastavi vrednost na 0, kadarkoli smo brez kovancev.


Vhodni podatki funkcije so torej $kovanci$ (seznam vseh vrednosti kovancev, ki so nam na voljo), $n$ (število kovancev), ter $sum$ (znesek denarja, za katerega iščemo število možnih kombinacij).

In [27]:
(* #31 Coin Sums *)

(*vrne i-ti element seznama*)
let rec i_ti lst i =
  match lst with
  | [] -> failwith "prazen seznam"
  | x :: xs -> if i = 0 then x else i_ti xs (i - 1)


let rec coin_sum kovanci n sum = match sum with
  (*oba robna pogoja*)
  | 0 -> 1
  | s -> if sum < 0 || n = 0 then 0 
  (*in rekurzivna formula*)
  else coin_sum kovanci n (s - i_ti kovanci (n - 1)) + coin_sum kovanci (n - 1) s


let rec st_kombinacij kovanci sum = 
  (*kličem funkcijo coin_sum na ustreznih vhodnih podatkih*)
  coin_sum kovanci (List.length kovanci) sum
 

val i_ti : 'a list -> int -> 'a = <fun>


val coin_sum : int list -> int -> int -> int = <fun>


val st_kombinacij : int list -> int -> int = <fun>


In [31]:
let resitev1 = st_kombinacij [1; 2; 5; 10; 20; 50; 100; 200] 200

val resitev1 : int = 73682


#### #117 Red, Green and Blue Tiles

In [59]:
let tiles dolzina_ploscice =
  (*dolzina_ploscice je celo število*)
  let st_moznosti = Array.make (dolzina_ploscice + 1) 0 in
  (*v arrayu shranjeno število možnosti za sestaviti ploščice dolžin 0, ... , n*)
  st_moznosti.(0) <- 1;
  (*robni primer - za ploščivo dolžine 0 samo ena možnost*)

  let rec aux i =
    (*indeks i predstavlja trenutno dolžino ploščice*)
    if i <= dolzina_ploscice then (
      (*rekurzivne formule - podrobneje razložene v tekstu spodaj*)
      st_moznosti.(i) <- (if i >= 1 then st_moznosti.(i - 1) else 0) +
                         (if i >= 2 then st_moznosti.(i - 2) else 0) +
                         (if i >= 3 then st_moznosti.(i - 3) else 0) +
                         (if i >= 4 then st_moznosti.(i - 4) else 0);
      (*rekurzivno se pomika vsakič za eno dolžino ploščice naprej*)                   
      aux (i + 1)
    )
  in
  (*začne z indeksom 1, indeks 0 ima že določeno vrednost*)
  aux 1;
  (*rezultat funkcije je število možnosti na zadnjem elementu v arrayu*)
  st_moznosti.(dolzina_ploscice)

val tiles : int -> int = <fun>


In [60]:
let primer1 = tiles 5

let resitev2 = tiles 50

val primer1 : int = 15


val resitev2 : int = 100808458960497


Rekurzivna formula za izračun števila možnosti torej izgleda takole: 
$\\ st\_moznosti(i) = \begin{cases} 1 & ;\text{if } i = 0 \\
st\_moznosti(i-1) + st\_moznosti(i-2) + st\_moznosti(i-3) + st\_moznosti(i-4) & ;\text{if } i > 0 \end{cases} \\$.

- Časovna zahtevnost:
funkcija rekurzivno kliče sama sebe $n$ - krat, kjer je $n$ dolžina ploščice, v funkciji označena kot $dolzina\_ploscice$, na vsakem koraku rekurzije pa porabi konstantno časa, zato je skupna časovna zahtevnost funkcije kar $O(n)$

- Prostorska zahtevnost:
funkcija gradi array $st\_moznosti$ dolžine $dolzina\_ploscice + 1$, zato je tudi prostorska zahtevnost enaka $O(dolzina\_ploscice) = O(n)$